Experimento:
Buscamos lograr contestar la pregunta:

¿Es posible asociar el valor de alguna característica de la música como su energía o su tempo con un lugar geográfico?


In [62]:
import pandas as pd
import numpy as np


#juntar paises en base al continente
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split

from sklearn.metrics import classification_report
from sklearn.preprocessing import StandardScaler

In [63]:
df = pd.read_excel('C:/Users/diego/OneDrive/Escritorio/universidad/semestre10/mineria/T2/Insight_Alchemist/Spotify.xlsx')

#Ajustamos columnas para contraarrestar error de formato en archivo xlsx
df['duration_ms'] = df['duration_ms']/10
df['popularity'] = df['popularity']/10
df['streams'] = df['streams']/10
df['af_danceability'] = df['af_danceability']/1000
df['af_energy'] = df['af_energy']/1000
df['af_key'] = df['af_key']/10
df['af_loudness'] = df['af_loudness']/1000
df['af_speechiness'] = df['af_speechiness']/1000
df['af_acousticness'] = df['af_acousticness']/1000
df['af_instrumentalness'] = df['af_instrumentalness']/1000
df['af_liveness'] = df['af_liveness']/1000
df['af_valence'] = df['af_valence']/1000
df['af_tempo'] = df['af_tempo']/1000
df['af_time_signature'] = df['af_time_signature']/10

Primero modificamos la información de la columna regiones para reducir las clases al agrupar los países por regiones geográficas más amplia.

In [64]:
d1 = [
    (['Morocco','South Africa'],'Africa'),
    (['Argentina','Bolivia','Brazil','Chile','Colombia','Costa Rica','Dominican Republic','Ecuador','El Salvador','Guatemala','Honduras','Mexico','Nicaragua','Panama','Paraguay','Peru','Uruguay'],'America_latina'),
    (['Canada','United States'],'America_norte'),
    (['Hong Kong','India','Indonesia','Japan','Malaysia','Philippines','Singapore','Taiwan','Thailand','Vietnam'],'Asia'),
    (['Andorra','Austria','Belgium','Bulgaria','Czech Republic','Denmark','Estonia','Finland','France','Germany','Greece','Hungary','Iceland','Ireland','Italy','Latvia','Lithuania','Luxembourg',
     'Netherlands','Norway','Poland','Portugal','Romania','Russia','Slovakia','Spain','Sweden','Switzerland','Ukraine','United Kingdom'],'Europa'),
    (['Australia','New Zealand'],'Oceania'),
    (['Egypt','Israel','Saudi Arabia','Turkey','United Arab Emirates'],'Oriente_medio')
]

def apply_etiqueta(elemento):
    for i in d1:
        if elemento in i[0]:
            return i[1]
        
new_reg = df["region"]


for index, row in df.iterrows():
    new_data = apply_etiqueta(row["region"])
    new_reg.iat[index] = new_data

#se reemplaza la columa de region
df["region"] = new_reg


Para encontrar si esta relación existe vamos a usar un Decision Tree Classifier.

Primero vamos atributo por atributo probando para ver cual tiene mejores resultados.

In [65]:
for i in ['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']:

    df = df.dropna(subset=["region", i])  # Drop rows where target or feature is NaN
    X = df[i].copy().values.reshape(-1, 1)
    y = df["region"]

    #primero separamos los datos de entrenamiento y validacion/test
    X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

    # Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
    X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

    #estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
    std_scaler = StandardScaler()
    X_train_std_scaled = std_scaler.fit_transform(X_train)

    X_val_std_scaled = std_scaler.transform(X_val)
    X_test_std_scaled = std_scaler.transform(X_test)

    #definimos nuestro modelo
    clf = DecisionTreeClassifier(criterion="gini",random_state=0)

    #entrenamos al modelo con fit
    clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

    #realizamos predicciones de nuestros datos con los datos de validación
    y_val_pred = clf.predict(X_val_std_scaled)


    #metricas de las predicciones
    #print(i)
    #print(classification_report(y_val, y_val_pred))

De lo anterior se pude observar que las dos características que presentan mayor relación son loudness y tempo.

Ahora se va a hacer un codigo que pruebe con todas las posibilidades

In [66]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

L = ['af_danceability','af_energy', 'af_key', 'af_loudness', 'af_mode', 'af_speechiness','af_acousticness', 'af_liveness', 'af_valence','af_tempo', 'af_time_signature']

# Se crea una lista con todas las combinaciónes posibles
print(len(L))
newList = []
def sublists(lst, new, index=0, current=[]):
    if index == len(lst):
        new.append(current)
        #print(current)
        return
    sublists(lst,new, index+1, current)
    sublists(lst,new, index+1, current + [lst[index]])

sublists(L,newList)

print(len(newList))
#sacamos los repetidos
unique_newList = []
for i in newList:
    if i not in unique_newList:
        unique_newList.append(i)

#sacamos el elemento vacio
unique_newList.remove([])
print(len(unique_newList))

#accuracy list
accuracy_list = []

for i in unique_newList:
    subset = ["region"] + i
    df = df.dropna(subset=subset)  # Drop rows where target or feature is NaN
    X = df[i].copy()
    y = df["region"]

    #primero separamos los datos de entrenamiento y validacion/test
    X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

    # Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
    X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

    #estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
    std_scaler = StandardScaler()
    X_train_std_scaled = std_scaler.fit_transform(X_train)

    X_val_std_scaled = std_scaler.transform(X_val)
    X_test_std_scaled = std_scaler.transform(X_test)

    #definimos nuestro modelo
    clf = DecisionTreeClassifier(criterion="gini",random_state=0)

    #entrenamos al modelo con fit
    clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

    #realizamos predicciones de nuestros datos con los datos de validación
    y_val_pred = clf.predict(X_val_std_scaled)


    #metricas de las predicciones

    accuracy = accuracy_score(y_val,y_val_pred)
    precision_micro = precision_score(y_val, y_val_pred, average='micro')
    precision_macro = precision_score(y_val, y_val_pred, average='macro')
    recall_micro = recall_score(y_val, y_val_pred, average='micro')
    recall_macro = recall_score(y_val, y_val_pred, average='macro')
    f1_micro = f1_score(y_val, y_val_pred, average='micro')
    f1_macro = f1_score(y_val, y_val_pred, average='macro')

    #para poder saber cual fue el mejor se va a sacar el promedio de todos los score y se van a guardar en una lista
    promedio = (accuracy + precision_micro + precision_macro + recall_micro + recall_macro + f1_micro + f1_macro)/7 

    accuracy_list.append((i,promedio))

#ordenamos la lista resultante
sorted_by_score = sorted(accuracy_list, key=lambda tup:tup[1])


11
2048
2047


c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\diego\OneDrive\Escritorio\universidad\semestre10\mineria\T2\Insight_Alchemist\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in la

Se imprimen las 10 peores y las 10 mejores conjuntos

In [69]:
print(sorted_by_score[:10])
print(sorted_by_score[2036:2046])

[(['af_time_signature'], np.float64(0.2864815037161335)), (['af_mode'], np.float64(0.2864815037161335)), (['af_mode', 'af_time_signature'], np.float64(0.2864815037161335)), (['af_key'], np.float64(0.2864815037161335)), (['af_key', 'af_mode'], np.float64(0.29640393600964965)), (['af_key', 'af_time_signature'], np.float64(0.31580236121712757)), (['af_key', 'af_mode', 'af_time_signature'], np.float64(0.33895782009387393)), (['af_liveness'], np.float64(0.4260734008648214)), (['af_acousticness'], np.float64(0.4377899173040535)), (['af_danceability'], np.float64(0.43925500120213584))]
[(['af_key', 'af_liveness', 'af_valence', 'af_tempo', 'af_time_signature'], np.float64(0.7016625443436151)), (['af_energy', 'af_key', 'af_loudness', 'af_acousticness', 'af_valence', 'af_tempo', 'af_time_signature'], np.float64(0.7016709033736281)), (['af_danceability', 'af_energy', 'af_key', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo'], np.float64(0.7016783483969826)), (['af_key', 'af_mode', '

Se predice usando los datos test con el mejor conjunto

In [70]:
df = df.dropna(subset=['region','af_danceability', 'af_energy', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo'])  # Drop rows where target or feature is NaN
X = df[['af_danceability', 'af_energy', 'af_mode', 'af_speechiness', 'af_acousticness', 'af_valence', 'af_tempo']].copy()
y = df["region"]

#primero separamos los datos de entrenamiento y validacion/test
X_train, X_val_and_test, y_train, y_val_and_test = train_test_split(X,y,test_size=0.3,random_state=0,stratify=y)

# Luego separamos los datos de validación y pruebas                                       0.5 x 0.3 = 0.15
X_val, X_test, y_val, y_test = train_test_split(X_val_and_test, y_val_and_test, test_size=0.5, random_state=0, stratify=y_val_and_test)

#estandarizacion de datos, transformar los datos de manera que tengan media 0 y desviacion estandar 1.
std_scaler = StandardScaler()
X_train_std_scaled = std_scaler.fit_transform(X_train)

X_val_std_scaled = std_scaler.transform(X_val)
X_test_std_scaled = std_scaler.transform(X_test)

#definimos nuestro modelo
clf = DecisionTreeClassifier(criterion="gini",random_state=0)

#entrenamos al modelo con fit
clf.fit(X_train_std_scaled, y_train)   ## Entrenar usando X (features), y (clase)

#realizamos predicciones de nuestros datos con los datos de validación
y_test_pred = clf.predict(X_test_std_scaled)


#metricas de las predicciones
print("todas")
print(classification_report(y_test, y_test_pred))


todas
                precision    recall  f1-score   support

        Africa       0.93      0.46      0.62      1785
America_latina       0.82      0.75      0.79     15151
 America_norte       0.64      0.17      0.27      1785
          Asia       0.88      0.67      0.76      8922
        Europa       0.68      0.92      0.78     25448
       Oceania       0.83      0.22      0.35      1785
 Oriente_medio       0.86      0.38      0.53      4461

      accuracy                           0.74     59337
     macro avg       0.80      0.51      0.58     59337
  weighted avg       0.77      0.74      0.73     59337

